# Classificador utilizando Naive Bayes

Para Naive Bayes, podemos utilizar duas modalidades de algorítmo:
- Multinomial Naive Bayes
- Bernoulli Naive Bayes

É recomendado que testemos ambas as possibilidades para garantir o melhor encaixe.

In [2]:
# importe bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

In [3]:
fileHandle = open('kw_list_train.txt','r') # esta lista possui as palavras-chave de cada link analisado
kw_train = fileHandle.readlines()
fileHandle.close()

fileHandle = open('target_train.txt','r') # esta lista corresponde aos valores-alvo ordenados de acordo com a lista anterior
target_train = fileHandle.readlines()
fileHandle.close()

# Para confirmar se nossos dados estão alinhados, o comprimento de cada lista deve ser o mesmo:
# Número de lista de palavras-chave == numero de resultados 0 ou 1
# Entendemos 0 como uma notícia não-pertinente, que não se trata de violência, e 1 como uma notícia sobre o uso
# violento de armas de fogo.
if (len(kw_train) == len(target_train)):
    print("O número de sites analisados está igual ao número de resultados! Ótimo!")

O número de sites analisados está igual ao número de resultados! Ótimo!


Precisei criar esta função abaixo para rearranjar o documento com a lista de listas de keywords para separar o resultado de cada website. A função resulta em uma lista consultável de 2 dimensões.

In [13]:
# Este bloco formata as strings dos arquivos lidos removendo caracteres indesejados.

kw_trainA = [] # Preferi criar uma lista vazia para manipular os dados. Caso dê algo errado nossa lista inicial estará à salvo.

# Como o resultado esperado é uma lista de listas, sendo a lista secundária uma str a ser analisada pelo NB, faz-se:
for line in kw_train:
    sublist = []
    line = line.split(",")
    for word in line:
        word = word.rstrip()
        word = word.strip(" ' \" \] \[ ") # tira aspas, colchetes e espaços
        sublist.append(word)
    sublist = str(sublist)
    kw_trainA.append(sublist)

targetA = []
for line in target_train:
    line = line.rstrip()
    targetA.append(line)


['preso', 'áudios', 'aragarças', 'família', 'tiros', 'jovem', 'suposta', 'pms', 'jeferson', 'troca', 'morreu', 'menino', 'polícia', 'apontam', 'carro']
<class 'list'>


# CRIANDO O DATAFRAME

Vamos criar um dataframe. É como se fosse uma tabela do bom e velho Excel e que vai nos dar mais facilidade na manipulação de dados.

In [14]:
data = {'KW': kw_trainA, 'TARGET': targetA} # as informações serão divididas em duas colunas,
                                            #cujos valores são as listas formatadas previamente.
news_df = pd.DataFrame(data, columns=['KW','TARGET'])

In [15]:
# Exibe 10 amostras aleatórias do dataset.
news_df.sample(10)

,KW,TARGET
136,"['provoca', 'foto', 'instagram', 'posta', 'hen...",0
19,"['assassinado', 'santos', 'edísio', 'descarta'...",1
75,"['morto', 'caldas', 'tiros', 'jovem', 'residên...",1
69,"['adolescente', 'tráfico', 'araçatuba', 'homem...",1
27,"['rosto', 'morto', 'pública', 'tiros', 'idoso'...",1
62,"['acusado', 'matar', 'suspeito', 'disparo', 'd...",1
134,"['arremessar', 'armas', 'seres', 'homens', 'te...",0
111,"['disparos', 'morto', 'crime', 'acionada', 'vi...",1
6,"['preso', 'tupã', 'porta', 'relacionamento', '...",1
87,"['pm', 'clonada', 'casa', 'filho', 'armas', 'm...",1


Para utilizar o próprio conjunto de treinamento para testar a acurácia, utilizaremos a ferramenta train_test_split.

In [16]:
from sklearn.model_selection import train_test_split

# a coluna KW será nosso 'X' enquanto a coluna TARGET será nosso 'y'

X_train, X_test, y_train, y_test = train_test_split(
    news_df['KW'], 
    news_df['TARGET'], 
    random_state = 1 # este random state serve para caso desejemos reproduzir a mesma pseudoaleatoriedade em outro momento
)

print("Training dataset: ", X_train.shape[0]) # o dataset de treinos possui esta dimensão
print("Test dataset: ", X_test.shape[0]) # e o dataset de teste possui esta dimensão.

Training dataset:  105
Test dataset:  35


Creio que a parte mais importante é a vetorização das palavras-chave. O modelo NB não aceita a leitura de caracteres diretamente, sendo então necessária a conversão de uma string de palavras para um vetor.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train) # vetorizaremos tanto o train set
testing_data = count_vector.transform(X_test) # quanto o test set

In [25]:
print(training_data) # é assim que se parecerá o set vetorizado
len(training_data)

  (0, 482)	1
  (0, 169)	1
  (0, 342)	1
  (0, 53)	1
  (0, 152)	1
  (0, 513)	1
  (0, 6)	1
  (1, 658)	1
  (1, 414)	1
  (1, 235)	1
  (1, 737)	1
  (1, 691)	1
  (1, 473)	1
  (1, 621)	1
  (1, 56)	1
  (1, 120)	1
  (1, 190)	1
  (1, 344)	1
  (1, 692)	1
  (1, 659)	1
  (1, 649)	1
  (1, 18)	1
  (2, 601)	1
  (2, 665)	1
  (2, 310)	1
  :	:
  (103, 586)	1
  (103, 422)	1
  (103, 289)	1
  (103, 706)	1
  (103, 526)	1
  (103, 666)	1
  (103, 498)	1
  (103, 199)	1
  (103, 692)	1
  (104, 455)	1
  (104, 40)	1
  (104, 275)	1
  (104, 69)	1
  (104, 594)	1
  (104, 718)	1
  (104, 36)	1
  (104, 562)	1
  (104, 614)	1
  (104, 589)	1
  (104, 556)	1
  (104, 300)	1
  (104, 19)	1
  (104, 503)	1
  (104, 190)	1
  (104, 53)	1


In [28]:
clf = MultinomialNB() # eis então o Multinomial Naive Bayes
clf.fit(training_data, y_train) # fit é o comando que recebe os dados de X e y para criar o classificador.

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
# Após treinado o modelo, a predição é feita com o seguinte comando:

predictions = clf.predict(testing_data)
predictions # exibe o resultado das predições.

array(['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0',
       '1', '1', '1', '0', '1', '1', '1', '1', '1'], dtype='<U1')

# RESULTADOS

É observado que o resultado está impreciso/viciado pela falta de exemplos negativos. Será necessário voltar à coleta de dados e reaplicar o treinamento no modelo para que este possua mais parâmetros comparativos.

In [31]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Recall score: ", recall_score(y_test, predictions, average = 'weighted'))
print("Precision score: ", precision_score(y_test, predictions, average = 'weighted'))
print("F1 score: ", f1_score(y_test, predictions, average = 'weighted'))

Accuracy score:  0.9142857142857143
Recall score:  0.9142857142857143
Precision score:  0.9017316017316018
F1 score:  0.9063736263736264
